In [1]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import skmob

## Reading the DataFrame

### Data original

In [3]:
path = '../../../DataTelkomsel/2021Desember/Desember2021.csv'
df = pd.read_csv(path)
df = df.drop({'Unnamed: 0'},axis=1)
df.head()

/tmp/ipykernel_31678/1340175577.py:2: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,maid,latitude,longitude,timestamp
0,b74fb520-8f92-47fe-9771-affc32c6af5d,-7.79426,110.36891,1638291600
1,b74fb520-8f92-47fe-9771-affc32c6af5d,-7.79424,110.36891,1638291600
2,b74fb520-8f92-47fe-9771-affc32c6af5d,-7.79424,110.36891,1638291600
3,b74fb520-8f92-47fe-9771-affc32c6af5d,-7.79426,110.36891,1638291600
4,8e8ab5b5-5bef-4e12-aa59-d460a97a13e1,-7.834843,110.3233,1638291600


### Data reverse geocoding 

In [4]:
path2 = '../../../DataTelkomsel/2021Desember/RGdesember2021.csv'
df2 = pd.read_csv(path2)
df2 = df2.drop({'Unnamed: 0'},axis=1)
df2.head()

,maid,latitude,longitude,datetime_wib,geometry,Kelurahan/Desa,Kecamatan,Kabupaten
0,b74fb520-8f92-47fe-9771-affc32c6af5d,-7.79426,110.368910,2021-12-01 00:00:00,POINT (110.36891 -7.79426),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta
1,b74fb520-8f92-47fe-9771-affc32c6af5d,-7.79424,110.368912,2021-12-01 00:00:00,POINT (110.3689117431641 -7.79424),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta
2,b74fb520-8f92-47fe-9771-affc32c6af5d,-7.79426,110.368912,2021-12-01 00:00:00,POINT (110.3689117431641 -7.79426),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta
3,b74fb520-8f92-47fe-9771-affc32c6af5d,-7.79425,110.368919,2021-12-01 00:00:00,POINT (110.3689193725586 -7.79425),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta
4,b74fb520-8f92-47fe-9771-affc32c6af5d,-7.79424,110.368912,2021-12-01 00:00:00,POINT (110.3689117431641 -7.79424),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta


## 1. Filter Unnecessary Area
Membatasi area data point pada range yg dilakukan analisis

In [8]:
def filter_area(gps, road, filename):
    gps_path = gps
    road_path = road
    
    tdf = skmob.TrajDataFrame.from_file(gps_path)
    area_shape = gpd.read_file(road_path)
    
    gdf_gps = GeoDataFrame(tdf, geometry=gpd.points_from_xy(tdf['longitude'], tdf['latitude']))
    filtered_data = gdf_gps[gdf_gps.geometry.within(area_shape.geometry.iloc[0])].copy()
    
    filtered_data['datetime_wib'] = pd.to_datetime(filtered_data['datetime_wib'])
    filtered_data['tanggal'] = filtered_data['datetime_wib'].dt.date
    
    filtered_data.to_csv(filename + '.csv', index=False)    

    filtered_data.info()

    print("Filtering area for ", filename, "succeed")

In [3]:
gps = df2
road = '../JaringanJalan/Malioboro_jalan/clipping_boundary.geojson'
filename ='filter1_malar_des.csv'
filter_area(gps, road, filename)

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 22038 entries, 0 to 51644
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   maid            22038 non-null  object        
 1   latitude        22038 non-null  float64       
 2   longitude       22038 non-null  float64       
 3   datetime_wib    22038 non-null  datetime64[ns]
 4   geometry        22038 non-null  geometry      
 5   Kelurahan/Desa  22038 non-null  object        
 6   Kecamatan       22038 non-null  object        
 7   Kabupaten       22038 non-null  object        
 8   tanggal         22038 non-null  object        
dtypes: datetime64[ns](1), float64(2), geometry(1), object(5)
memory usage: 1.7+ MB
Filtering area for  filter2_maljan_jan succeed


In [10]:
data = pd.read_csv(f'./{filename}.csv')
data.head()

,Unnamed: 0,maid,latitude,longitude,datetime_wib,geometry,Kelurahan/Desa,Kecamatan,Kabupaten,tanggal
0,39,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.794847,110.365608,2021-12-01 00:00:35,POINT (110.36560821533205 -7.794847),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-01
1,102,3ad1fad6-400a-4ae9-8610-c2d355dcd9f9,-7.794029,110.365944,2021-12-01 00:01:24,POINT (110.3659439086914 -7.794029),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-01
2,103,3ad1fad6-400a-4ae9-8610-c2d355dcd9f9,-7.794029,110.365944,2021-12-01 00:01:24,POINT (110.3659439086914 -7.794029),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-01
3,104,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.794847,110.365608,2021-12-01 00:01:27,POINT (110.36560821533205 -7.794847),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-01
4,105,d60a2068-d7ea-48a7-a363-e3e7e0bd02cb,-7.794847,110.365608,2021-12-01 00:01:27,POINT (110.36560821533205 -7.794847),Kelurahan Suryatmajan,Danurejan,Kota Yogyakarta,2021-12-01


In [ ]:
data.nunique()

Unnamed: 0.1      597265
Unnamed: 0        597265
maid               13106
latitude            9314
longitude            837
datetime_wib      103084
geometry           58483
Kelurahan/Desa         5
Kecamatan              4
Kabupaten              1
tanggal               30
dtype: int64